In [19]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df_fake = pd.read_csv("/content/drive/MyDrive/Fake.csv")
df_true = pd.read_csv("/content/drive/MyDrive/True.csv")


In [21]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [22]:
df_fake["class"] = 0
df_true["class"] = 1

In [23]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)


df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [24]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-24-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-24-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [25]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [26]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [27]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [28]:
df.isnull().sum()

,0
text,0
class,0


In [29]:
df = df.sample(frac = 1)



In [30]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [31]:
df.head()

,text,class
0,MSNBC anchor Joe Scarborough was caught on a h...,0
1,KUALA LUMPUR (Reuters) - Malaysian police thwa...,1
2,BOSTON/WASHINGTON (Reuters) - Conservative and...,1
3,Obama promised to keep his SOTU speech short b...,0
4,"On Sunday, Rudy Giuliani appeared on Fox News ...",0


In [32]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AdamW
from torch.optim.lr_scheduler import StepLR

In [33]:
# Subset the dataset to 10k samples
df = df.head(1000)

# Tokenizer from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom dataset class to preprocess data for BERT
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        # Tokenize the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create the dataset
max_length = 128
train_dataset = NewsDataset(df['text'].values, df['class'].values, tokenizer, max_length)

# Create a DataLoader with a smaller batch size (e.g., 8)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Initialize BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Training function
def train_model(model, train_loader, optimizer, epochs=2):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, optimizer, epochs=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [34]:
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader

# Define the dataset for testing/evaluation
#test_df = pd.read_csv('your_test_data.csv')  # Replace with your actual test dataset
test_df = df_fake_manual_testing.head(10)

test_dataset = NewsDataset(test_df['text'].values, test_df['class'].values, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Put the model in evaluation mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate classification report (precision, recall, f1-score, etc.)
print(classification_report(all_labels, all_preds))


              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.00      0.00      0.00         0

    accuracy                           0.80        10
   macro avg       0.50      0.40      0.44        10
weighted avg       1.00      0.80      0.89        10



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [42]:
API_KEY = "Your api key"
CX_KEY = "Your cse id"

In [41]:
import torch
import torch.nn.functional as F
import requests
import gradio as gr
import spacy
from sentence_transformers import SentenceTransformer, util

# Load SpaCy for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Load SBERT Model for Semantic Similarity Check
sbert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Define trusted sources
TRUSTED_SOURCES = [
    # General & National News (India)
    "ndtv.com", "timesofindia.indiatimes.com", "thehindu.com", "hindustantimes.com",
    "indianexpress.com", "news18.com", "firstpost.com", "thequint.com",
    "scroll.in", "deccanherald.com", "tribuneindia.com", "dnaindia.com",
    "telegraphindia.com", "outlookindia.com", "theweek.in", "newslaundry.com",
    "theprint.in", "opindia.com",

    # Business & Finance News (India)
    "economictimes.indiatimes.com", "business-standard.com", "livemint.com",
    "moneycontrol.com", "financialexpress.com", "cnbctv18.com",

    # Technology & Science News (India)
    "gadgets360.com", "tech.hindustantimes.com", "indiatoday.in/technology",
    "digit.in", "thestatesman.com",

    # Regional News (India)
    "manoramaonline.com", "oneindia.com", "anandabazar.com", "sakshi.com",
    "jagran.com", "amarujala.com", "lokmat.com", "dainikbhaskar.com",
    "dailythanthi.com", "eenadu.net", "mathrubhumi.com", "prajasakti.com",

    # Political & Opinion Journalism (India)
    "swarajyamag.com", "thewire.in", "caravanmagazine.in", "thelogicalindian.com",

    # Sports News (India)
    "espncricinfo.com", "cricbuzz.com", "sportstar.thehindu.com",

    # Entertainment & Bollywood News (India)
    "filmfare.com", "bollywoodhungama.com", "pinkvilla.com", "koimoi.com"
]


# Function to fetch news articles from Google Search API
def fact_check_with_api(text, api_key, cx_key):
    url = f"https://www.googleapis.com/customsearch/v1?q={text}&key={api_key}&cx={cx_key}"
    response = requests.get(url)
    data = response.json()

    verified_sources = []
    if "items" in data:
        for item in data["items"]:
            if any(source in item["link"] for source in TRUSTED_SOURCES):
                verified_sources.append((item["title"], item["link"]))
    return verified_sources

# Function to extract key entities from a text
def extract_entities(text):
    doc = nlp(text)
    return {ent.text for ent in doc.ents}

# Function to check similarity between input news and verified articles
def check_similarity(input_text, verified_sources):
    input_embedding = sbert_model.encode(input_text, convert_to_tensor=True)
    max_similarity = 0
    best_match = None

    for title, link in verified_sources:
        title_embedding = sbert_model.encode(title, convert_to_tensor=True)
        similarity_score = util.pytorch_cos_sim(input_embedding, title_embedding).item()
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            best_match = (title, link)

    return max_similarity, best_match

# Function to classify news using the fine-tuned BERT model
def predict_with_bert(text):
    encoding = tokenizer.encode_plus(text, return_tensors="pt", padding="max_length", max_length=128, truncation=True)
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)  # Convert logits to probabilities
        prediction = torch.argmax(probs, dim=1).item()

    labels = ["Misinformation", "Reliable"]
    confidence = probs[0][prediction].item()  # Get confidence score

    return labels[prediction], confidence

# Function to combine API, SBERT, and BERT results
def analyze_news(text, api_key, cx_key):
    # Step 1: Check API for trusted sources
    verified_sources = fact_check_with_api(text, api_key, cx_key)

    if verified_sources:
        similarity_score, best_match = check_similarity(text, verified_sources)

        if similarity_score > 0.7:  # Lowered threshold for better accuracy
            return f"✅ Verified as True (Similarity: {similarity_score:.2f})\nBest Source: {best_match[0]} - {best_match[1]}"

    # Step 2: Use fine-tuned BERT model
    bert_prediction, bert_confidence = predict_with_bert(text)

    # Step 3: Final Decision - Combine All Outputs
    if verified_sources and similarity_score > 0.5:
        final_decision = f"✅ Likely True (API & Similarity: {similarity_score:.2f})"
    elif bert_prediction == "Reliable" and bert_confidence > 0.7:
        final_decision = f"✅ Likely True (BERT Confidence: {bert_confidence:.2f})"
    else:
        final_decision = f"❌ Likely Misinformation (BERT & No Trusted Source)"

    return final_decision

# Gradio Interface
def gradio_interface(text):
    return analyze_news(text, API_KEY, CX_KEY)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=["text"],
    outputs="text",
    title="Misinformation Detection and Fact-Checking"
)

# Launch the Gradio interface
iface.launch()


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f1137d7cac3c0beec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
